# Setup Cookie for NSDIC Login

## Notebook Purpose

Running this Notebook will create a cookie file and create or append a `~.netrc` for use by *gdal* and *QGIS* to remotely access files at NSIDC server. 

This file should work for days to weeks and can be refreshed by running this notebook again. In particular, this Notebook should be used for *QGIS* projects with remote links to files at NSIDC. 

The setup process for this notebook also applies to several other notebooks for working with GIMP data.

## Getting Started

The following instructions assume the presence of several python packages. The instructions below have been tested with a conda (Anaconda) install, which is the recommended approach especially for novice python users.  The conda-specic instructions should contain sufficient information for users proficient with other package managers (e.g., pip) to get this notebook up and running.

### GIMP Specific Code

As a first step, users will need to download some custom code from git hub. This step can be performed by either cloning the repository https://github.com/fastice/gimpfunc.git from browser or from the command line via:

`git clone git@github.com:fastice/gimpfunc.git`

For simplicity, this step can either be excuted in the directory where you wish to run the notebook. Better yet install it somewhere central and add it to your `PYTHONPATH` environment variable. Or uncomment and modify as needed this command so that the directory is accessible to this Notebook:

In [1]:
# By default this command assumes gimpfunc is under the home directory. Modify with approprite path as needed.
# import sys
# import os
# sys.path.append('os.path.expanduser('~/gimpfunc'))

### Conda Install

For users not already using conda, this package manager can be downloaded from: https://www.anaconda.com/products/individual. Unfortunately, some of the standard anaconda releases do not work well with this notebook and its strongly reccomended that all packages be downloaded from the conda-forge channel. Because, mixing conda-forge with the regular conda packages can cause some package conflict issues, the cleanest approach is to create a new conda environment. (For those unfamiliar with conda, this means creating an environment that when activated runs a completely independent set of code). Using a seperate enviroment will both help ensure success in running this notebook as well avoiding causing problems with existing conda environments (users with a conda-forge priority environment can probably use their existing environment).

The `gimpfunc` directory downloaded from github contains a file, `environment.yaml` that will create a new environment called `forge` by executing the command: 

`conda env create -f environment.yaml`

This command will create a new a new environment and download the listed packages and their dependencies, totalling about 1GB. Some of these packages are not required for this notebook, but will be required by other GIMP notebooks. Everytime this notebook is run, activate the environment via:

`conda activate forge` (Note the name `forge` can be changed prior to creating the evironment by editing the `environment.yaml` file.)

Juptyer lab can then be run from the command line (ideally from the directory where the notebook is located) as:

`jupyter lab NISDCLoginNotebook.ipynb`

### Trouble Shooting

The above conda installation should ensure that the version of jupyter lab is >= 3.0. If an earlier version of jupyter lab is used, then the following extensions need to be enabled:
* `jupyter labextension install @pyviz/jupyterlab_pyviz`
* `jupyter labextension install @jupyter-widgets/jupyterlab-manager`
* `jupyter labextension install @jupyterlab/server-proxy`

Note without the extendsions nothing may appear broken(i.e., no error message), but the Notebook likely will not function correctly.

## Setup

This next cell will import everything need for the login. If something above was not installed properly, its likely an error will occur at this stage.

In [2]:
import gimpfunc.NASALogin as NASALogin
import panel
import os
panel.extension()

Adding /home/ian/anaconda3/share/qgis/python to sys.path


## Create the Login Cookie and update/create .netrc File

### What this Process Will Do

Many of the GIMP notebooks and QGIS projects use remote links to data archived at NSIDC. Each a progam (e.g., *QGIS*) or Notebook requests these data it needs to authenticate via the user's NASA EarthData login (users who don't have a login can sign up here: https://urs.earthdata.nasa.gov/users/new). This Notebook will set up that authentication so that the process happens automatically when a Notebook or QGIS try to access those files. In particular, it will create a cookie file (`.gimp_download_cookiejar.txt`) in the user's home directory that *GDAL* (one of the installed packages) uses to redirect the login appropriately for remote file access. It also creates a new or appends to an existing `.netrc` file in the user's home directory to include the login id and password (see security risks below). In principle, this notebook should only need to be run once to set thes files up, but can be rerun if needed. Further steps outlined below are required for programs to use this information.

If a valid cookie file and .netrc file already exists, this step will print a simple text message saying so and no further action is needed. If not, a widget will allow entry of NASA Earth Data credentials. If the login is successful, the status message will turn blue and indicate success. Orange indicates the cookie file exists, but login credentials are needed for the .netrc file.

In [3]:
myLogin = NASALogin()
#myLogin.resetCookie() # Uncomment to delete prior cookie (e.g., if expired or corrupted)
myLogin.view()

Row
    [0] ParamMethod(method)
    [1] Column(margin=5, name='Earth Data Login', width=300)
        [0] StaticText(value='<b>Earth Data Login</b>')
        [1] TextInput(name='Username')
        [2] PasswordInput(name='Password')
        [3] Button(name='Enter Credentials')
    [2] ParamMethod(method)

### Troubleshooting

If it appears that the credentials were entered correctly, but the status message does not indicate so, run the following cell, which will indicate login success as `True` and failure as `False`. It also checks whether a .netrc file exists. If still having problems, check the contents of the .netrc file.

In [6]:
print(f'Login status: {myLogin.check_cookie()}')
print(f'.netrc exists: {os.path.exists(os.path.expanduser("~/.netrc"))}')

Login status: True
.netrc exists: True


## Potential Security Risks

The `.netrc` file that is created as part of this process is a potential security risk as it saves your username/password (only NASA Earth Data Login in this case) as clear text. The file is created as user only access (-rw-------). If your acount is reasonably secure and you are using a password **_unique_** to this login, then this may be a reasonable risk (i.e, only your NASA Earth Data account could be compromised). If you are concerned, delete `~/.netrc` after each use and then regenerate with this notebook as needed.

## QGIS Setup

If a QGIS project includes remote */vsicurl* links to remote products at NSIDC GDAL needs to authenticate using the cookie file just described. So QGIS needs to know how to find it. Depending on QGIS starts, it may get these directly from the users enviroment variables. In which case all the user has to do is specify them. For example,
* `export GDAL_HTTP_COOKIEFILE=~/.gimp_download_cookiejar.txt` for bash or `setenv GDAL_HTTP_COOKIEFILE ~/.gimp_download_cookiejar.txt` for csh/tcsh.
* `export GDAL_HTTP_COOKIEJAR=~/.gimp_download_cookiejar.txt` for bash or `setenv GDAL_HTTP_COOKIEJAR ~/.gimp_download_cookiejar.txt` for csh/tcsh.

Alternatively, you can set the variables in QGIS as `Settings->Options->System->Environment` - see https://github.com/fastice/greenland-ice-mapping-project/blob/main/QGIS.md.

Note at least with conda installed *QGIS* called from an Ubuntu command line, the data load much quicker when the cookie is specified with user specified environment variables (as opposed to inside QGIS). As a result, it may be a good choice to select the `If Undefined` option under the Apply column when you enter the variables in QGIS. This will cause to use the system variables as the first and fastest option when they are available.

### QGIS Troubleshooting

The setup procedure has been verified to work with 'QGIS' V3.16 and above. It did not work with at least one instance of V3.10. 

In building the environment, a version of QGIS was installed, which while it can be opened via a command line call on Ubuntu 20.04 it does not appear to open properly on Mac Big Sur. Thus, some users will have greater success downloading and installing the app directly (it should also be more full featured than the conda version). Some of the notebooks, however, need to load the `QGIS` core libraries, which can be difficult to find when it is installed as an app. Thus, users should make sure the conda version is also installed.

## Other Notebooks

The login setup in this notebook is required for some of the other notebooks. Alternatively, these can be set as environment variables as described above under **QGIS Setup**.

By default the cookie file will reside in the users home directory with the name `.gimp_download_cookiejar.txt`. To point other notebooks to the cookie file within a notebook, use a suitably modifed version of the command below. 

In [5]:
env = dict(GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/.gimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/.gimp_download_cookiejar.txt'))
os.environ.update(env)